# Load packages and configuration

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from equilibrator_api import ComponentContribution, Q_ #for creating quantity variables
import equilibrator_api
import equilibrator_custom_functions_paper as eq #custom functions created during analysis
import numpy as np #numerical package
import math
import matplotlib.pyplot as plt #plotting package
import custom_plot_functions as cpf

import pandas as pd

# if more space is required
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll {height: 100em; }</style>"))

# set defaults for plotting - grid on, linewidth = 2
plt.rc( 'axes', grid=True  )
plt.rc( 'figure', figsize = (7,4), dpi=96)
plt.rc( 'axes', linewidth=1 )
plt.rc( 'lines', linewidth=2 )

# Define physiological conditions and metabolite ranges

In [ ]:
#E. coli physiological conditions as a dictionary
E_coli_con = {
            "p_h": str(Q_("7.5")),
            "p_mg": str(Q_(6)),
            "ionic_strength": str(Q_('200 mM')),
            "temperature": str(Q_(37+273.15,"K")),
            "kcat_source": "fwd",
}
cs_default_bounds = eq.obtain_compound_settings("metabolite_reference_table_expanded_custom_phys_conc_paper", custom_bounds = True)

cc= ComponentContribution()

# Compute MDFs

In [ ]:
%%time

results_excel= ''
xls = pd.ExcelFile(results_excel+'.xlsx')
sheets = xls.sheet_names[1:]
xls.close()

mdf_simulations_all={}

for sheet in sheets:
    print(sheet)
    # Perform MDF Simulation
    MDF_result = eq.MDF_analysis(results_excel,sheet,cs_default_bounds,E_coli_con, y=0, custom_dGs = False, print_results = True)
    MDF,res1,g1,g2,g3,g4,flx,shp_r,r_id,res2,met,bl,bu,c,shp_c = eq.print_MDF_results_individual(MDF_result)

    # Save results in dictionary
    mdf_simulations_all[sheet] = [MDF,MDF_result.reaction_df, MDF_result.compound_df, res1,g1,g2,g3,g4,flx,shp_r,r_id,res2,met,bl,bu,c,shp_c] 
    

# Save results in Excel

In [ ]:
# Store all MDF scores in an Excel file 

MDF_score_dict={}

for sheet_MDF in sheets:
    MDF_score_dict[sheet_MDF]=np.round(mdf_simulations_all[sheet_MDF][0],3)
    
MDF_score_df=pd.DataFrame.from_dict(MDF_score_dict, orient='index')
MDF_score_df.columns=['MDF Score (kJ/mol)']

filename= 'yourfilename.xlsx'

with pd.ExcelWriter(filename, mode="w", engine="openpyxl") as writer:
    MDF_score_df.to_excel(writer, sheet_name='Summary', encoding='UTF-8')

    
# Store all data and graphs from each MDF calculation in an independent sheet
import openpyxl

for sheet_MDF_data in sheets:
    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists ='overlay') as writer:
        
        # Save reaction information
        react_df= mdf_simulations_all[sheet_MDF_data][1]
        react_df.to_excel(writer,sheet_name=sheet_MDF_data,startrow=0 , startcol=0)   
        
        # Save metabolite information
        metab_df= mdf_simulations_all[sheet_MDF_data][2]
        metab_df.to_excel(writer,sheet_name=sheet_MDF_data,startrow=(len(react_df))+3 , startcol=0)   


# Create plot with all simulations

In [ ]:
# Create plot with all simulations

import math 
# integrate in one figure
fig, axs = plt.subplots(math.ceil((len(mdf_simulations_all.keys()))/3),3, figsize=(30,40), facecolor="white" )

ind = 0

for mdf_sim in mdf_simulations_all.keys():
    
    # Extract all data necessary for the plot from the raw simulation data
    MDF = mdf_simulations_all[mdf_sim][0]
    g3 = mdf_simulations_all[mdf_sim][6]
    g4 = mdf_simulations_all[mdf_sim][7]
    r_id = mdf_simulations_all[mdf_sim][10]
    shp_r = mdf_simulations_all[mdf_sim][9]

    # Set the position of the graph for every for loop
    vert_indx= math.floor(ind/3)
    lat_indx= ind -3*vert_indx
    
    # Plot and edit the graph
    cpf.plot_bar_chart( MDF,g3,g4,r_id,shp_r, ax=axs[vert_indx,lat_indx] )
    axs[vert_indx,lat_indx].set_title(mdf_sim +' = '+ str(np.round(MDF,3)) + ' kJ/mol', fontsize=10)
    axs[vert_indx,lat_indx].set_ylim( [-60, 40] )
    axs[vert_indx,lat_indx].tick_params( axis='x', rotation=90 )
    axs[vert_indx,lat_indx].legend(loc=3)
    ind = ind+1
        #cpf.plot_concentrations_my(MDF_result)  
        #display(res1)
        #display(res2)

fig.tight_layout()
fig.show()
        
        



# Create plots for individual simulations

In [ ]:
from importlib import reload
reload(cpf)

import openpyxl

wb = openpyxl.load_workbook(filename)        

for sheet_MDF_data in wb.sheetnames[1:]:
        # Create and save MDF graph
        
        # Extract all data necessary for the plot from the raw simulation data
        MDF = mdf_simulations_all[sheet_MDF_data][0]
        g3 = mdf_simulations_all[sheet_MDF_data][6]
        g4 = mdf_simulations_all[sheet_MDF_data][7]
        r_id = mdf_simulations_all[sheet_MDF_data][10]
        shp_r = mdf_simulations_all[sheet_MDF_data][9]
 
        # Plot and edit the graph
        cpf.plot_bar_chart( sheet_MDF_data,MDF,g3,g4,r_id,shp_r)
                
        # Save graph
        graph_name= sheet_MDF_data + ' MDF Reaction Plot.png' 
        plt.savefig(graph_name, bbox_inches='tight')